<h2>IMPORTS</h2>

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import sklearn
from sklearn.cluster import KMeans
from matplotlib.colors import LinearSegmentedColormap

pozycje = np.array(np.genfromtxt("pozycje.csv", delimiter=',')) #array containing positions of sources obtained from FieldTrip
inside = np.array(np.genfromtxt("inside.csv", delimiter=',')) #array of inside sources labels
elecpos = np.array(np.genfromtxt("elecpos.csv", delimiter=',')) #array of electrode positions
M = np.array(np.genfromtxt("Macierz_przejscia.csv", delimiter=','))
Z1 = np.array(np.genfromtxt("Z_1.csv", delimiter=','))
Z2 = np.array(np.genfromtxt("Z_2.csv", delimiter=','))
Z3 = np.array(np.genfromtxt("Z_3.csv", delimiter=','))
H3 = np.array(np.genfromtxt("H_3.csv", delimiter=','))
H2 = np.matmul(Z3, H3)
H1 = np.matmul(Z2, H2)
Mp = np.matmul(Z1, H1)

<h2>Calculations</h2>

<h3>Positions of inside sources</h3>

In [ ]:
temp = []
for i in range(len(inside)):
    if inside[i] == 1:
        temp.append(pozycje[i])
poz_ins = np.array(temp)

<h3>plot sources</h3>

In [ ]:
x = poz_ins[:,0]
y = poz_ins[:,1]
z = poz_ins[:,2]

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.view_init(elev=10, azim=10)

plt.xlabel("X")
plt.ylabel("Y")
ax.set_zlabel("Z")

ax.set_xlim(-0.1,0.1)
ax.set_ylim(-0.1,0.1)
ax.set_zlim(-0.06,0.1)

#ax.scatter(x, y, z, marker='.', color=(0.5,0.5,0.5))

v=1
r = 0
for xi,yi,zi in zip(x,y,z):
    ax.scatter(xi, yi, zi, marker='${}$'.format(v), color=(r,r,r))
    r+= 0.0007
    
#plt.savefig("myImage.png", format="png", dpi=1200)
plt.show()

<h3>plot electrodes</h3>

In [ ]:
xe = poz_ins[:,0]
ye = poz_ins[:,1]
ze = poz_ins[:,2]

fig = plt.figure()
ax = fig.add_subplot()
plt.xlabel("X")
plt.ylabel("Y")
ax.set_xlim(-0.12,0.15)
ax.set_ylim(-0.12,0.15)

v=1
for xi,yi,zi in zip(xe,ye,ze):
    ax.scatter(yi, zi, marker='${}$'.format(v), color=(0.5,0.5,0.5))
    v+=1
#plt.savefig("myImage.png", format="png", dpi=1200)
plt.show()

<h2>clustering</h2>

<p>preparation of data considering every element of M as a datapoint</p>

In [ ]:
Hvec = M.flatten()
Hvec_norm = np.linalg.norm(Hvec)
Hvec /= Hvec_norm
Hl = []
for i in Hvec:
  Hl.append([1,i])
Hl = np.array(Hl)

c = 4
kmeans = KMeans(n_clusters=c)

<p>Preparation of data considering an electric potential vector (along 3 axies) of each source as a datapoint</p>

In [ ]:
mt = M
Hl = np.reshape(mt, (-1, 3))
print(np.shape(Hl))

c = 4
kmeans = KMeans(n_clusters=c)

<h3>source clustering</h3>

In [ ]:
kmvec = kmeans.fit_predict(Hl)
unique, counts = np.unique(kmvec, return_counts=True)

print(dict(zip(unique, counts)))

bl= np.reshape(kmvec, (128, 3930)) #choose the correct matrix size

<h2>HEATMAP</h2>

<h3>aligning the axies of source and electrode positions</h3>

In [188]:
#axies#
Z = 2
Y = 1
X = 0

macierz_s = bl.T
ref = poz_ins

#sort Z#
Zs = macierz_s[ref[:, Z].argsort()]
pozins_Zs = ref[ref[:, Z].argsort()]

#sort Y for each Z#
Zfragments_p =[]
Zfragments =[]
l1=[]

[l1.append(i) for i in ref[:,Z] if i not in l1]

for j in l1:
    matrr_f = np.array([list(i) for i,r in zip(macierz_s,ref) if r[Z]==j])
    fragment1 = np.array([list(i) for i in ref if i[Z]==j])
    Ys = matrr_f[fragment1[:,Y].argsort()]
    pozins_Ys = fragment1[fragment1[:,Y].argsort()]
    Zfragments.append(Ys)
    Zfragments_p.append(pozins_Ys)

#sort X for each Y#
ZYfragments=[]
ZYfragments_p=[]

for j,c in zip(Zfragments_p, Zfragments):
    Yfragments_p=[]
    Yfragments=[]
    l1=[]
    [l1.append(i) for i in j[:,Y] if i not in l1]

    for g in l1:
        matrr_f = np.array([list(i) for i,r in zip(c,j) if r[Y]==g])
        fragment1 = np.array([list(i) for i in j if i[Y]==g])
        Xs = matrr_f[fragment1[:,X].argsort()]
        pozins_Xs = fragment1[fragment1[:,X].argsort()]
        Yfragments_p.append(pozins_Xs)
        Yfragments.append(Xs)

    ZYfragments_p.append(Yfragments_p)
    ZYfragments.append(Yfragments)

matr = np.array([macierz_s[0]])

for i in ZYfragments:
    for j in i:
        matr = np.concatenate((matr, j), axis=0)
        
out = np.reshape(np.delete(matr, 0 ,0), np.shape(bl.T))

<h3>temp holds values of a single cluster</h3>

In [83]:
klaster = 0 #cluster nr
nx = 3930
mx = 3930
temp = np.zeros((nx,mx))

for i in range(nx):
    for j in range(mx):
        if bl[i][j] == klaster: #bl (raw) or out.T (aligned axies)
            temp[i][j] = 1

<h3>get heatmap</h3>

In [ ]:
Z_align_axis = bl#[elecpos[:, 2].argsort()] #only for 128xM matrix, ssort along chosen axie
                #bl or out.T

myColors = ((0.0, 0.0, 0.0, 1.0), (0.0, 0.0, 0.7, 1.0), (0.7, 0.0, 0.0, 1.0), (0.9, 0.9, 0.9, 1.0))
cmap = LinearSegmentedColormap.from_list('Custom', myColors, len(myColors))

ax = sns.heatmap(Z_align_axis, cmap=cmap,  cbar_kws={'label': 'numer klastra'})# cbar=False)
ax.set(xlabel='numer żródła', ylabel='numer cechy')

colorbar = ax.collections[0].colorbar
colorbar.set_ticks([0.36, 1.133, 1.86, 2.63])
colorbar.set_ticklabels(['1', '2', '3', '4'])

#plt.savefig("./zdjecia/Z1.png", format="png", dpi=600, bbox_inches="tight")
plt.show()

<h3>get heatmap for a single cluster</h3>

In [ ]:
Z_align_axis = temp#[elecpos[:, 2].argsort()] #only for 128xM matrix

ax = sns.heatmap(temp, cbar=False)
ax.set(xlabel='numer żródła', ylabel='numer cechy')

#plt.savefig("./zdjecia/Z1Z2Z3.png", format="png", dpi=600, bbox_inches="tight")
plt.show()